#### 0. Get retranslation sentences

In [ ]:
from Modules import *
from TrainingUtils import *
import pickle
import os

In [ ]:
model_path = "data/transformer_cn.bin"
vocab_path = "data/vocab_cn.pkl"
spcial_dict_path = "data/dict6000.pkl"
input_data_path = ""
data_save_path = ""

In [ ]:
def vocab_segment(str1,vocab):
    str2 = []
    i=0
    while i< len(str1):
        if str1[i] in vocab.keys():
            refer_vocab = vocab[str1[i]]
            view_window = len(max(refer_vocab, key=len)) + i#找出包含该字词典
            if view_window >len(str1):
                view_window = len(str1)
            use_flag = False
            while view_window >= i+1:
                if str1[i:view_window] in refer_vocab:
                    str2.append("".join(str1[i:view_window]))
                    use_flag= True
                    break
                else:view_window -= 1
            if use_flag == True:
                i = view_window
            else:
                str2.append(str1[i])
                i = i + 1
        else:
            str2.append(str1[i])
            i = i +1
    return str2

def load_model(path,vocab_path):
    params = torch.load(path, map_location=lambda storage, loc: storage)
    state_dict = params['state_dict']

    SRC,TGT = pickle.load(open(vocab_path, "rb"))


    model = make_model(SRC,TGT, N=6,d_model=512, d_ff=2048, h=8,dropout=0.1)
    model.load_state_dict(state_dict)

    model = model.cuda()
    model = model.eval()
    return model,SRC,TGT

def interactive(line,model,SRC,TGT):
    vocab = pickle.load(open(special_dict_path, "rb"))
    line = "".join(line.split())
    line = vocab_segment(line,vocab)
    line = [word for word in " ".join(line)]
    line = [SRC.stoi[w] for w in line]
    src_len = len(line)
    src_sent = torch.LongTensor([line]).cuda()
    src_mask = torch.ones(1, 1, src_len).cuda()
    return line_greedy_decode(100,model, src_sent, src_mask,TGT.stoi["<s>"],TGT)

In [ ]:
model,SRC,TGT = load_model(model_path, vocab_path)

In [ ]:
input_data = datareading(input_data_path)
output = []
for line in input_data:
    output.append(interactive(line,model,SRC,TGT))

In [ ]:
savefiletxt(output,data_save_path)

#### 1. File input

In [1]:
import pickle as pkl

def eachFile(filepath):
    child = []
    filename = []
    pathDir = os.listdir(filepath)
    pathDir = sorted(pathDir)
    for allDir in pathDir:
        child.append(
            os.path.join('%s%s' % (filepath, allDir)))
        filename.append(allDir)
    return filename,child

def datareading(filepath):
    text = []
    file = open(filepath, "r",encoding="utf-8")
    for line in file.readlines():
        text.append("".join(line.strip().split()))
        
    text = [[word for word in line] for line in text]
    text = [" ".join(line) for line in text]
    return text

def savefiletxt(text,filepath):
    f = open(filepath,"w",encoding='utf-8')
    for i in text:
        f.write(str(i))
        f.write('\n')
    f.close()

In [2]:
gu = datareading("AMCT/trans27.1_an.txt") # Model retranslation output
bai = datareading("AMCT/trans27.1_cn.txt") # Model output

In [3]:
gu_refer = datareading("AMCT/trans_27.1_an_refer.txt") # Ancient Chinese reference
bai_refer = datareading("AMCT/trans_27.1_cn_refer.txt") # Modern Chinese reference

#### 2. Dual BLEU computing

In [4]:
from evaluation_utils import *
from bleu import *

In [5]:
def sentences_bleu(refers,outputs):
    bleu_scores = []
    for i in range(len(refers)) :
        refer = refers[i]
        output = outputs[i]
        bleu, precisions, bp, ratio, translation_length, reference_length = compute_bleu(
            [[refer]],[output],max_order=2,smooth=True)
        bleu_scores.append(bleu)
    return bleu_scores

In [6]:
dual_bleu_score = sentences_bleu(gu_refer,gu)

#### 3. Average BLEU

In [8]:
def average_sentences_bleu(refers,outputs):
    bleu_scores = []
    for i in range(len(refers)) :
        refer = refers[i]
        bleu_score = 0.
        for j in range(len(outputs)):
            output = outputs[j]
            bleu, precisions, bp, ratio, translation_length, reference_length = compute_bleu(
                [[refer]],[output],max_order=2,smooth=True)
            bleu_score += bleu
        bleu_scores.append(bleu_score/len(refers))
    return bleu_scores

In [ ]:
bleu_average = average_sentences_bleu(gu_refer,gu)

#### 4. Sentence  similarity

In [ ]:
def cos_sim(vectors_a, vectors_b):
    sims = []
    for i in range(len(vectors_a)):
        vector_a = np.mat(vectors_a[i])
        vector_b = np.mat(vectors_b[i])
        num = float(vector_a * vector_b.T)
        denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
        cos = num / denom
        sim = 0.5 + 0.5 * cos
        sims.append(sim)
    return sims

def average_embed(text,vocab_dict):
    text = [line.split()for line in text]    
    sentence_embedding = []
    for i in range(len(text)):
        line_embeding = np.full((512), 0, dtype="float32")
        for j in range(len(text[i])):
            line_embeding += vocab_dict[text[i][j]][0]
        sentence_embedding.append(list(line_embeding/[len(text[i])]))
    return sentence_embedding

In [ ]:
embed_dict=pickle.load(open("order/emb_word2vec_an.pkl", "rb"))

In [ ]:
gu_refer_w2c = average_embed(gu_refer,embed_dict)
gu_w2c =  average_embed(gu,embed_dict)
sent_sim = cos_sim(gu_w2c, gu_refer_w2c)

#### 5. DTE computing

In [ ]:
def score_normal(score):
    score_mean = np.mean(score)  
    score_std = np.std(score)
    score_normal = []
    for i in range(len(score)):
        score_normal.append((float(score[i]) - score_mean)/score_std)
    return score_normal

In [ ]:
dual_normal = score_normal(dual_bleu_score)
avg_bleu_normal = score_normal(average_sentences_bleu)
sent_sim_normal = score_normal(sent_sim)

In [ ]:
DTE_sent_score = dual_bleu_normal + avg_bleu_normal + sent_sim_normal

In [ ]:
DTE_corpus_score = np.mean(DTE_sent_score)